In [1]:
# import preprocess as prep
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
import importlib
import preprocess
import SLR_model
from scipy. stats import mode
# model will output multiple(15) results/sec, how are we gonna handle it?

2024-11-20 10:09:01.372692: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-20 10:09:01.404765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-20 10:09:01.414772: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-20 10:09:01.417774: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 10:09:01.450679: I tensorflow/core/platform/cpu_feature_guar

In [2]:
importlib.reload(SLR_model)
importlib.reload(preprocess)


<module 'preprocess' from '/home/shd/BomNae-SLR/preprocess.py'>

In [3]:
# Configurations
model_dir = "saves"
THRESHOLD = 0.5
# batch_size = 16

In [5]:
# 세이브된 모델 로드
load_path = os.path.join(model_dir, "ckpt","11-19-19-30epochs-5times_final.keras")
model = SLR_model.load_model(load_path)

In [6]:
# load data
l_raws=[]
r_raws=[]
p_raws=[]
y_raws=[]
person_id = 18
# TODO hardcode path
# a video
for j in sorted(os.listdir(os.path.join(preprocess.getoutputdir('val'),str(person_id)))):
    j="0162.npz"
    l_raw, r_raw, p_raw, y_raw = preprocess.load_data(f"{person_id}/{j}",'val')
    
    l_raws.append(l_raw)
    r_raws.append(r_raw)
    p_raws.append(p_raw)
    y_raws.append(y_raw)
    l_test, each = SLR_model.serialize(l_raws)
    r_test, each = SLR_model.serialize(r_raws)
    p_test, each = SLR_model.serialize(p_raws, stride=2,is_pose = True)
    x_test = (l_test, r_test, p_test)
    print(x_test[0].shape)
    print(x_test[1].shape)
    print(x_test[2].shape)
    break
    

(9, 63, 4, 5, 3)
(9, 63, 4, 5, 3)
(9, 32, 10, 3)


In [7]:
# load data
l_raws=[]
r_raws=[]
p_raws=[]
y_raws=[]
person_id = 17
count=0
# TODO hardcode path
# a video
val_list=sorted(os.listdir(os.path.join(preprocess.getoutputdir('val'),str(person_id))))
for j in val_list:
    # j="0001.npz"
    try:
        l_raw, r_raw, p_raw, y_raw = preprocess.load_data(f"{person_id}/{j}",'val')
    except Exception as e:
        print(e)
        continue
    l_raws.append(l_raw)
    r_raws.append(r_raw)
    p_raws.append(p_raw)
    y_raws.append(y_raw)
    l_test, each = SLR_model.serialize(l_raws)
    r_test, each = SLR_model.serialize(r_raws)
    p_test, each = SLR_model.serialize(p_raws, stride=2,is_pose = True)
    x_test = (l_test, r_test, p_test)
    try:
        predictions = model.predict(x_test,32,verbose=0)
        ans=SLR_model.decode_onehot2d(predictions)
        num=mode(ans)
        
    except Exception as e:
        log_msg=f'{j}) {e}\n'
    else:
        log_msg=f'{j}) 답:{y_raw}/pred:{num[0]}\n'
        if int(y_raw)==int(num[0]):
            count+=1
    finally:
        with open("test_log.txt","a") as log:
            log.write(log_msg)
        l_raws.clear()
        r_raws.clear()
        p_raws.clear()
        y_raws.clear()
print(count)
print(count/len(val_list)*100)

I0000 00:00:1732065000.760296  226857 service.cc:146] XLA service 0x72ada800e360 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732065000.760316  226857 service.cc:154]   StreamExecutor device (0): Host, Default Version
2024-11-20 10:10:00.787548: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732065001.029115  226857 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-11-20 10:10:13.814045: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/home/shd/.conda/envs/BomNae-SLR/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You

KeyboardInterrupt: 

In [7]:
y_raws

[160]

In [8]:
# # dummy x_raw data
# l_raws = []
# duration_list = []
# h_elem = np.zeros(shape=(4,5,3))
# # p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = (np.random.randint(90,100))
# duration_list.append(duration)
# for j in range(duration): # frame size
#     x_elem.append(h_elem)
#     # x_elem.append(comb_elem)
# l_raws.append(x_elem)
# r_raws = []
# 
# h_elem = np.zeros(shape=(4,5,3))
# # p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = duration_list[0]
# for j in range(duration): # frame size
#     x_elem.append(h_elem)
#     # x_elem.append(comb_elem)
# r_raws.append(x_elem)
# p_raws = []
# 
# # h_elem = np.zeros(shape=(4,5,3))
# p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = duration_list[0]
# for j in range(duration): # frame size
#     # x_elem.append(h_elem)
#     x_elem.append(p_elem)
#     # x_elem.append(comb_elem)
# p_raws.append(x_elem)

In [9]:
l_test, each = SLR_model.serialize(l_raws)
r_test, each = SLR_model.serialize(r_raws)
p_test, each = SLR_model.serialize(p_raws, stride=2,is_pose = True)
x_test = (l_test, r_test, p_test)

# dataset = SLR_model.convert_to_dataset_single(x_test, batch_size)


# with open(hist_path, 'w') as file:
#     json.dump(hist.history, file)

In [10]:
x_test[2].shape

(9, 32, 10, 3)

In [11]:

predictions = model.predict(x_test,32)
print(SLR_model.decode_onehot2d(predictions))


I0000 00:00:1731986886.251456  215083 service.cc:146] XLA service 0x735938002f90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731986886.251472  215083 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2024-11-19 12:28:06.259792: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-19 12:28:06.281038: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 715ms/step
tf.Tensor([ 160 1061  160  160  160  160  160  160  160], shape=(9,), dtype=int64)


I0000 00:00:1731986886.861900  215083 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [12]:
predictions

array([[8.8778928e-28, 1.0076214e-35, 0.0000000e+00, ..., 1.6910694e-28,
        1.9434814e-25, 8.0171348e-30],
       [2.2881200e-39, 1.7597240e-39, 0.0000000e+00, ..., 3.0345118e-40,
        1.0116219e-35, 8.2298259e-42],
       [5.5863837e-36, 1.2805408e-37, 0.0000000e+00, ..., 1.2908472e-34,
        1.0130329e-31, 1.3706373e-37],
       ...,
       [1.1347761e-28, 2.8873146e-35, 0.0000000e+00, ..., 2.2443479e-29,
        2.5087139e-28, 3.3000588e-28],
       [3.4661793e-31, 2.5240023e-38, 0.0000000e+00, ..., 3.1294363e-32,
        1.8734681e-32, 6.1250624e-31],
       [5.0636566e-26, 1.9238760e-34, 2.2476827e-42, ..., 1.1359085e-27,
        7.3203604e-29, 8.8434659e-27]], dtype=float32)

In [ ]:
# pred_bin = predictions.round().astype(np.int32)
# pred_bin

In [ ]:
importlib.reload(SLR_model)

In [ ]:
SLR_model.decode_onehot2d(predictions)

In [ ]:
SLR_model.bin2num([1,0,1,1,1,0,1,1,1,0,1,0])

In [ ]:
model.predict